In [10]:
import csv
import cv2
import numpy as np
import datetime
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [11]:
local = './Data/data/driving_log.csv'
img_loc = './Data/data/IMG/'
file_loc = './Data/data/driving_log.csv'
data_loc = './Data/data/'

In [12]:
lines = []

#load csv file
with open(local) as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)
    for line in reader:
        lines.append(line)

In [13]:
def get_measurement(sample, i):
    correction_factor = 0.2
    # print(sample[3])
    if i == 0:
        return float(sample[3])
    elif i == 1:
        return float(sample[3]) + correction_factor
    elif i == 2:
        return float(sample[3]) - correction_factor
    return None

In [14]:
angles = []
image_paths = []
with open(file_loc) as csvfile:
    reader = csv.reader(csvfile)
    strt = next(reader)
    i = 0
    for line in reader:
        line.append(file_loc.split("/")[0])
        for i in range(0, 3):
            angle = get_measurement(line, i)
            angles.append(angle)
            split_values = line[i].split("/")
            source_path = img_loc + split_values[-1]
            image_paths.append(source_path)

In [15]:
#Make training and Validation data set and apply preprocessing

X_train = []
for path in image_paths:
    img = cv2.imread(path)
    shape = img.shape
    crop_up = int(shape[0]/5)
    crop_down = shape[0]-25
    img = img[crop_up:crop_down, 0:shape[1]]
    img = cv2.resize(img, (200, 66),  
               interpolation = cv2.INTER_AREA) 
    X_train.append(img)


In [16]:
print(X_train[0].shape)
print(len(X_train))

(66, 200, 3)
24108


In [17]:
import pandas as pd
import os
data_df = pd.read_csv(os.path.join(os.getcwd(), data_loc, 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

X = data_df[['center', 'left', 'right']].values
y = data_df['steering'].values
print(len(X))
X_train1, X_valid1, y_train1, y_valid1 = train_test_split(X, y, test_size=0.2, random_state=0)
print(len(X_train1))
print(len(y_train1))

8037
6429
6429


In [18]:
from sklearn.utils import shuffle
shuffle(np.array(X_train), np.array(angles))
X_train = np.asarray(X_train)
angles = np.asarray(angles)

In [19]:
#image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
#train_data_gen = image_gen.flow_from_directory(batch_size=32,
 #                                              directory=data_loc,
 #                                              shuffle=True,
 #                                              target_size=(66, 200))

In [20]:
X_train, X_valid = train_test_split(image_paths, test_size = 0.2)

In [23]:
#aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
#                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
#                         horizontal_flip=True, fill_mode="nearest")
print(X_train1[0])
print(y_valid1[0])

['IMG/center_2016_12_01_13_44_51_459.jpg'
 ' IMG/left_2016_12_01_13_44_51_459.jpg'
 ' IMG/right_2016_12_01_13_44_51_459.jpg']
 -0.2496443


In [1]:

import cv2, os
import numpy as np
import matplotlib.image as mpimg


IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)


def load_image(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))


def crop(image):
    """
    Crop the image (removing the sky at the top and the car front at the bottom)
    """
    return image[60:-25, :, :] # remove the sky and the car front


def resize(image):
    """
    Resize the image to the input shape used by the network model
    """
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)


def rgb2yuv(image):
    """
    Convert the image from RGB to YUV (This is what the NVIDIA model does)
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)


def preprocess(image):
    """
    Combine all preprocess functions into one
    """
    image = crop(image)
    image = resize(image)
    image = rgb2yuv(image)
    return image


def choose_image(data_dir, center, left, right, steering_angle):
    """
    Randomly choose an image from the center, left or right, and adjust
    the steering angle.
    """
    choice = np.random.choice(3)
    if str(steering_angle) == False:
        if choice == 0:
            return load_image(data_dir, left), steering_angle + 0.2
        elif choice == 1:
            return load_image(data_dir, right), steering_angle - 0.2
        return load_image(data_dir, center), steering_angle


def random_flip(image, steering_angle):
    """
    Randomly flipt the image left <-> right, and adjust the steering angle.
    """
    if np.random.rand() < 0.5:
        if str(steering_angle) == False:
            image = cv2.flip(image, 1)
            random_flip = -steering_angle
    return image, steering_angle


def random_translate(image, steering_angle, range_x, range_y):
    """
    Randomly shift the image virtially and horizontally (translation).
    """
    trans_x = range_x * (np.random.rand() - 0.5)
    trans_y = range_y * (np.random.rand() - 0.5)
    print(steering_angle)
    steering_angle += trans_x * 0.002
    trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, trans_m, (width, height))
    return image, steering_angle


def random_shadow(image):
    """
    Generates and adds random shadow
    """
    # (x1, y1) and (x2, y2) forms a line
    # xm, ym gives all the locations of the image
    x1, y1 = IMAGE_WIDTH * np.random.rand(), 0
    x2, y2 = IMAGE_WIDTH * np.random.rand(), IMAGE_HEIGHT
    xm, ym = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]

    # mathematically speaking, we want to set 1 below the line and zero otherwise
    # Our coordinate is up side down.  So, the above the line: 
    # (ym-y1)/(xm-x1) > (y2-y1)/(x2-x1)
    # as x2 == x1 causes zero-division problem, we'll write it in the below form:
    # (ym-y1)*(x2-x1) - (y2-y1)*(xm-x1) > 0
    mask = np.zeros_like(image[:, :, 1])
    mask[(ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0] = 1

    # choose which side should have shadow and adjust saturation
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    # adjust Saturation in HLS(Hue, Light, Saturation)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)


def random_brightness(image):
    """
    Randomly adjust brightness of the image.
    """
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


def augument(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    """
    Generate an augumented image and adjust steering angle.
    (The steering angle is associated with the center image)
    """
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    #image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    #image = random_shadow(image)
    image = random_brightness(image)
    return image, steering_angle


def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    """
    Generate training image give image paths and associated steering angles
    """
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            if str(steering_angle) == True:
                continue
            # argumentation
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augument(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center) 
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers

In [2]:
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential

from keras import backend as K
from keras.layers.core import Flatten, Dense, Lambda
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils
from keras.layers import Convolution2D, ELU,Conv2D,MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.layers.convolutional import Cropping2D

In [58]:
model = Sequential()
model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(66,200,3)))
model.add(Conv2D(24,(5, 5)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Conv2D(36, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Conv2D(48, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model.add(Flatten())
model.add(Dropout(0.2))
#model.add(Dense(1164, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='relu'))
model.summary()

#model.compile(optimizer=Adam(0.0001), loss="mse", )
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 62, 196, 24)       1824      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 61, 195, 24)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 57, 191, 36)       21636     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 190, 36)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 52, 186, 48)       43248     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 51, 185, 48)      

In [3]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only='true',
                                 mode='auto')
model.fit_generator(batch_generator(data_loc, X_train1, y_train1, 32, True),
                        steps_per_epoch = len(X_train1)*4,
                        epochs = 1,
                        shuffle = True,
                        validation_data=batch_generator(data_loc, X_valid1, y_valid1, 32, False),
                        validation_steps=len(X_valid1),
                        callbacks=[checkpoint],
                        verbose=1)

NameError: name 'model' is not defined

In [ ]:
model.fit_generator(train_data_gen, 
                    steps_per_epoch = len(X_train)*4, 
                    epochs = 2)
                    



In [17]:
history = model.fit(X_train, angles, epochs=1)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'str'>"}), <class 'numpy.ndarray'>

In [ ]:
model.save('model.h5')

In [19]:
import PIL


In [ ]:
import matplotlib.image as mpimg
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
def PreProcessImage(img):
    shape = img.shape
    crop_up = int(shape[0]/5)
    crop_down = shape[0]-25
    img = img[crop_up:crop_down, 0:shape[1]]
    img = cv2.resize(img, (200, 66),  
               interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    return img

def load_image(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))

def choose_image(data_dir, center, left, right, steering_angle):
    """
    Randomly choose an image from the center, left or right, and adjust
    the steering angle.
    """
    if str(center) == True:
        return
    choice = np.random.choice(3)
    if str(steering_angle) == False:
        if choice == 0:
            return load_image(data_dir, left), steering_angle + 0.2
        elif choice == 1:
            return load_image(data_dir, right), steering_angle - 0.2
        return load_image(data_dir, center), steering_angle

def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            if str(steering_angle) == True:
                continue
            if str(center) == True:
                continue
            if center == 'center':
                return
            image = load_image(data_dir,center)
            # add the image and steering angle to the batch
            images[i] = PreProcessImage(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers